In [77]:
import pandas as pd
import os
import numpy as np
import altair as alt

### Effective way of reading large datasets.

[Source](https://www.kaggle.com/kunalkotian/easily-load-train-csv-w-o-crash-save-feather-file)

In [2]:
!head -2 Data/train.csv > small_train.csv

In [3]:
small_train = pd.read_csv('small_train.csv')
print(small_train)

   id        date  store_nbr  item_nbr  unit_sales  onpromotion
0   0  2013-01-01         25    103665         7.0          NaN


In [ ]:
# types_dict = small_train.dtypes.to_dict()


In [4]:
types_dict = {'id': 'uint32',
             'item_nbr': 'uint32',
             'store_nbr': 'uint8',
             'unit_sales': 'float32'}

In [5]:
train = pd.read_csv('Data/train.csv', low_memory=True, dtype=types_dict)

/Users/eitharelbasheer/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
os.makedirs('Data/tmp', exist_ok=True)  # Make a temp dir for storing the feather file
# Save feather file, requires pandas 0.20.0 at least:
train.to_feather('Data/tmp/train_raw')

In [9]:
test = pd.read_csv('Data/test.csv', low_memory=True, dtype=types_dict)

In [10]:
test.to_feather('Data/tmp/test_raw')

In [41]:
train = pd.read_feather("Data/tmp/train_raw")

In [42]:
test = pd.read_feather("Data/tmp/test_raw")

## EDA

> Checking the memory consumption 

In [43]:
(train.memory_usage(index=True).sum())/1024**2

2534.3312063217163

In [44]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91635819 entries, 0 to 91635818
Data columns (total 6 columns):
id             uint32
date           object
store_nbr      uint8
item_nbr       uint32
unit_sales     float32
onpromotion    object
dtypes: float32(1), object(2), uint32(2), uint8(1)
memory usage: 2.5+ GB


In [45]:
(test.memory_usage(index=True).sum())/1024**2

57.85797119140625

In [63]:
EDA_df = train.copy()

* In pandas Date datatype is not vectorized, I will split it to the data column into three columns.

In [64]:
EDA_df['date'] = pd.to_datetime(train['date'],format="%Y-%m-%d")

In [65]:
EDA_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91635819 entries, 0 to 91635818
Data columns (total 6 columns):
id             uint32
date           datetime64[ns]
store_nbr      uint8
item_nbr       uint32
unit_sales     float32
onpromotion    object
dtypes: datetime64[ns](1), float32(1), object(1), uint32(2), uint8(1)
memory usage: 2.5+ GB


In [66]:
EDA_df['Year'] = pd.DatetimeIndex(EDA_df['date']).year
EDA_df['Month'] = pd.DatetimeIndex(EDA_df['date']).month
EDA_df['Day'] =pd.DatetimeIndex(EDA_df['date']).day.astype(np.uint8)

In [67]:
EDA_df

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,Year,Month,Day
0,0,2013-01-01,25,103665,7.0,None,2013,1,1
1,1,2013-01-01,25,105574,1.0,None,2013,1,1
2,2,2013-01-01,25,105575,2.0,None,2013,1,1
3,3,2013-01-01,25,108079,1.0,None,2013,1,1
4,4,2013-01-01,25,108701,1.0,None,2013,1,1
...,...,...,...,...,...,...,...,...,...
91635814,91635814,2016-09-21,30,1246784,4.0,False,2016,9,21
91635815,91635815,2016-09-21,30,1247036,2.0,False,2016,9,21
91635816,91635816,2016-09-21,30,1254013,1.0,False,2016,9,21
91635817,91635817,2016-09-21,30,1260238,4.0,False,2016,9,21


In [68]:
del(EDA_df['date'])

In [69]:
EDA_df['Day']=EDA_df['Day'].astype(np.uint8)
EDA_df['Month']=EDA_df['Month'].astype(np.uint8)
EDA_df['Year']=EDA_df['Year'].astype(np.uint16)

In [70]:
(EDA_df.memory_usage(index=True).sum())/1024**2

2184.768298149109

In [71]:
EDA_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91635819 entries, 0 to 91635818
Data columns (total 8 columns):
id             uint32
store_nbr      uint8
item_nbr       uint32
unit_sales     float32
onpromotion    object
Year           uint16
Month          uint8
Day            uint8
dtypes: float32(1), object(1), uint16(1), uint32(2), uint8(3)
memory usage: 2.1+ GB


In [72]:
# No need for column id for the analysis

EDA_df = EDA_df.drop(columns = "id")

* A clear reduction in the memory space.

In [74]:
EDA_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91635819 entries, 0 to 91635818
Data columns (total 7 columns):
store_nbr      uint8
item_nbr       uint32
unit_sales     float32
onpromotion    object
Year           uint16
Month          uint8
Day            uint8
dtypes: float32(1), object(1), uint16(1), uint32(1), uint8(3)
memory usage: 1.8+ GB


In [78]:
EDA_df.describe()

,store_nbr,item_nbr,unit_sales,Year,Month,Day
count,9.163582e+07,9.163582e+07,9.163582e+07,9.163582e+07,9.163582e+07,9.163582e+07
mean,2.730062e+01,9.069823e+05,6.078307e+00,2.014675e+03,6.368498e+00,1.561380e+01
std,1.636572e+01,4.827444e+05,2.102362e+01,1.060644e+00,3.338468e+00,8.798389e+00
min,1.000000e+00,9.699500e+04,-1.537200e+04,2.013000e+03,1.000000e+00,1.000000e+00
25%,1.100000e+01,5.023310e+05,2.000000e+00,2.014000e+03,4.000000e+00,8.000000e+00
50%,2.800000e+01,8.973480e+05,4.000000e+00,2.015000e+03,6.000000e+00,1.600000e+01
75%,4.300000e+01,1.239855e+06,9.000000e+00,2.016000e+03,9.000000e+00,2.300000e+01
max,5.400000e+01,2.037487e+06,4.414200e+04,2.016000e+03,1.200000e+01,3.100000e+01


* 54 stores
* Some items are not included in the training dataset.
* No observations with Unit sales for 0.

In [ ]:
EDA_df